### 执行上下文

执行第一行之前：  
* 变量会提前  
* 函数声明为提前
* this arguments 确定

In [1]:
fn();
//这是全局函数，提到了最前, 因此为正确执行
function fn(){ //函数声明会被提前
    console.log('声明');
}

声明


In [2]:
fn1(); //fn1此时为undefined; 
var fn1 = function(){ //这是函数表达式
    console.log('表达式');
}

TypeError: fn1 is not a function

In [3]:
// var a = undefined 
console.log(a); //此前有一步 var a = undefined; 
var a = 100;

undefined


In [5]:
//全局环境
fn('zhangsan')
function fn(name){ //提前到全局最前
    age = 20 
    console.log(name, age)
    var age //提前到函数体最前
    
    bar(100)
    function bar(num){ //提前到函数体前端
        console.log(num)
    }
}

zhangsan 20
100


### this  
this 要在执行时才能确认值，定义时无法确认     
arguments也是如此    

In [6]:
fn2('zhangsan')
function fn2(name){
    console.log(this);
    console.log(arguments);
}

Object [global] {
  global: [Circular],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] { [Symbol(util.promisify.custom)]: [Function] },
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(util.promisify.custom)]: [Function]
  },
  __filename: '[eval]',
  exports: {},
  module: Module {
    id: '[eval]',
    path: '.',
    exports: {},
    parent: undefined,
    filename: '/mnt/e/codes/Test/jsDemo/[eval]',
    loaded: false,
    children: [],
    paths: [
      '/mnt/e/codes/Test/jsDemo/node_modules',
      '/mnt/e/codes/Test/node_modules',
      '/mnt/e/codes/node_modules',
      '/mnt/e/node_modules',
      '/mnt/node_modules',
      '/node_modules'
    ]
  },
  __dirname: '.',
  require: [Function: require] {
    resolve: [Function: resolve] { paths: [Function: paths] },
    m

this作为对象属性执行；this就是对象

In [5]:
var a = {
    name:'A',
    fn: function (){
        console.log(this.name); //此时this是无法确定的；
    }
}
a.fn();//this === a， 是个对象

A


In [6]:
a.fn.call({name:'B'}) ; // this === {name:'B'}，重新绑定了

B


作为普通函数执行

In [7]:
var fn1 = a.fn;
fn1(); // this === window ,在nodejs下无定义； 

undefined


In [11]:
function fn3(){
    console.log(this); //
}
fn3;

[Function: fn3]

#### this在构造函数中

In [7]:
function Foo(name){
    //this = {}
    this.name = name;
    console.log(this)
    //return this;
}
var f = new Foo('zhangsan');

Foo { name: 'zhangsan' }


In [9]:
function fn(){
    console.log(this)  // 无返回，window
}

call apply bind

In [12]:
function fn4(name, age){
    console.log(name);
    console.log(this);
}
//指定第一个参数为this,第二个参数为函数第一个参数；
fn4.call({x:100},'zhangsan',20);

zhangsan
{ x: 100 }


In [14]:
fn4.apply({y:200},['lisi',30]);//apply, 以数组的方式传递参数； 

lisi
{ y: 200 }


bind 必须以函数表达式方式运行

In [15]:
var fn5 = function (name,age){ //需要用var,只能用函数表达式来bind
    console.log(name);
    console.log(this);
}.bind({z:300});
fn5('wangwu',40);

wangwu
{ z: 300 }


call最常用

### 作用域  
JS 没有块级作用域    
只有函数和全局作用域


In [10]:
if (true){
    var name = 'zhangsan'; //作用域与写在外面是一样的。推荐变量声明在外面；
}
console.log(name); // java, c++中会报错

zhangsan


In [17]:
var a = 100; //全局作用域，可能被覆盖。
function fn(){
    var a = 200; //函数作用域；
    console.log('fn',a);
}
console.log('global',a);
fn();

global 100
fn 200


作用域链   
当前作用域没有定义的变量，自由变量   
自由变量找定义的过程， 作用域链  

In [18]:
var a = 100;
function fn(){
    var b = 200;
    console.log(a);//当前作用域没有定义的变量，自由变量
    console.log(b);
}
fn();

100
200


函数的父级作用域，在函数定义时确定的  
与执行的地方无关；

In [11]:
var a = 100 
function F1(){
    var b = 200 
    function F2() {
        var c = 300 
        console.log(a) //与定义有关，写执行无关
        console.log(b)
        console.log(c)
    }
    F2()
}
F1()

100
200
300


作用域链：自由变量不断在父级作用域中查找变量声明；

### 闭包


In [19]:
function F1(){
    var a = 100;
    //函数的父级作用域是定义时的作用域，与执行时无关； 
    return function(){
        console.log(a); //a的父级作用域是F1();
    }
}
var f1 = F1(); //返回一个函数，f1就是函数了。
var a = 200; //全局作用域；
f1(); // 结果 中 a 只与定义的父级作用域有关

100


闭包的使用场景：  
* 函数作为返回值；函数中的自由变量只与定义时的作用域有关
* 函数作用参数传递；

In [21]:
function F2(fn){
    var a = 300;
    fn();
}
F2(f1); //a的父级作用域还是F1();与执行时无关；

100


In [ ]:
var i, a
for(i =0 ;i<10;i++){
    a = document.createElement('a');
    a.innerHTML = i + '<br>';
    a.addEventListener('click',function(e){
        var k = i; //无块级作用域，也无法正常工作
        e.preventDefault();
        alert(k);
    })
    document.body.appendChild(a);
}

In [ ]:
var i
for(i =0 ;i<10;i++){
    (function(i){ //自执行函数 
        var a = document.createElement('a');
        a.innerHTML = i + '<br>';
        a.addEventListener('click',function(e){ //e 是传入的event 
            e.preventDefault();
            alert(i);
        })
        document.body.appendChild(a);
    })(i) //传入函数作用域变量 i 
}

In [12]:
// 闭包的例子，共用函数内部变量。
// 主要用于封装变量，收敛权限。
function isFirstLoad(){
    var _list = [];
    return function(id){
        if(_list.indexOf(id)>=0){
            return false;
        } else {
            _list.push(id);
            return true;
        }
    }
}
var firstLoad = isFirstLoad();
firstLoad(10);

true

In [13]:
firstLoad(10);

false

In [14]:
var secondload = isFirstLoad(); // 新的返回函数，使用了新的内存空间。 
secondload(10);

true

In [15]:
secondload(10)

false

 你在isFirstLoad的函数外面，根本不可能修改_list 的值  
 函数的每次调用，共享一部分数据；